# Chapter 3 에서 배울 내용

1. **데이터 로더 제작**  
   - 모델 학습을 위한 데이터 로더 설계 및 구현  
   - 분자 데이터 분류를 위한 사전 지식 이해

---
2. **모델 제작**  
    - 분자 특성 분류/회귀 예측을 위한 모델 아키텍처 설계 
    - MLP, CNN, RNN, GNN 등 다양한 모델 활용 방법  
    - 입력 데이터 처리 및 모델에 맞는 출력 형식 설정

---
3. **학습 및 평가**
   - 모델 학습을 위한 손실 함수와 최적화 알고리즘 설정  
   - 교차 검증, 정확도, 정밀도, 재현율, F1 스코어 등 다양한 평가 지표 활용  
   - 과적합 방지를 위한 기법 (예: Dropout, 정규화, 데이터 증강 등)
   - 모델 성능 평가 
    
---
4. **결과 저장**
    - 학습 로그 및 평가 결과 기록  
    - 결과 시각화 및 분석 도구 사용 (예: Loss/Accuracy 그래프, confusion matrix 등)
    - 모델을 실험적 환경에서 재사용할 수 있도록 파일 포맷으로 저장 (예: `.pt`, `.h5` 등) 
    
---

In [3]:
import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU
from torch_geometric.data import Batch
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, GINConv, GATConv, global_max_pool as gmp, global_add_pool as gap

from tabulate import tabulate
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score

In [4]:
import warnings 
warnings.filterwarnings('ignore')

from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

## 1) 데이터 로더 제작

데이터 로더(Data Loader)는 머신러닝/딥러닝 모델 학습에서 데이터를 효율적으로 불러오고 처리하기 위한 도구다. 

일반적으로 `PyTorch`와 같은 프레임워크에서 제공되며, 대규모 데이터셋을 다룰 때 데이터를 배치 단위로 나누거나 전처리 작업을 자동화하는 데 사용된다.

데이터 로더를 사용하는 이유는 크게 다음과 같다.

---
1. **미니배치 생성**  
   - 데이터를 미니배치 단위로 나누어 처리하여 학습 속도와 메모리 효율성을 높임
   - 배치 크기가 작으면 메모리 사용량이 적으나, 학습 시간이 길어질 수 있음
   - 배치 크기가 크면 학습 속도가 빨라질 수 있으나, 메모리 사용량이 증가함

2. **데이터 셔플링**  
   - 학습 데이터 순서를 매 에포크마다 변경해 모델의 일반화 성능을 향상
   - 학습 시 특정 순서나 패턴으로 인해 모델이 편향되거나 과적합되지 않도록 하기 위해 사용됨.
   - 검증 및 테스트 데이터에서는 **shuffle을 사용하지 않음** (결과 재현성을 위해)

In [5]:
def loader_dataset(data_list, batch_size, shuffle=False):
    """
    DataLoader로 변경
    """
    collate = Batch.from_data_list(data_list)
    loader = DataLoader(data_list, batch_size=batch_size, collate_fn=collate, shuffle=shuffle)

    return loader

본 튜토리얼에서는 ESOL 데이터셋을 예시로 사용한다.

먼저, Chapter2에서 저장한 데이터들을 불러와 data loader를 생성한다.

In [6]:
def get_dataloader(dataset_name, data_type='Graph', batch_size=256):
    data_path = f'dataset/{dataset_name}/processed/'
    train_dataset = torch.load(os.path.join(data_path, f'{dataset_name}_{data_type}_train.pt'))
    valid_dataset = torch.load(os.path.join(data_path, f'{dataset_name}_{data_type}_valid.pt'))
    test_dataset = torch.load(os.path.join(data_path, f'{dataset_name}_{data_type}_test.pt'))    

    if data_type == 'Descriptors':  # Descriptors 데이터를 처리할 경우 scaling 필요
        # Step 1: Descriptors 데이터에서 x 값들만 추출
        raw_features_train = [data.x.view(-1).numpy() for data in train_dataset]
        raw_features_valid = [data.x.view(-1).numpy() for data in valid_dataset]
        raw_features_test = [data.x.view(-1).numpy() for data in test_dataset]
        # print(raw_features_train.shape)

        # Step 2: 스케일링을 위한 StandardScaler 적용
        # scaler = StandardScaler()
        scaler = MinMaxScaler()
        scaled_features_train = scaler.fit_transform(raw_features_train)  # 학습 데이터에 맞춰 스케일링
        scaled_features_valid = scaler.transform(raw_features_valid)  # 검증 데이터는 fit하지 않고 transform만 적용
        scaled_features_test = scaler.transform(raw_features_test)  # 테스트 데이터도 동일

        # Step 3: 스케일링된 데이터를 원본 데이터셋에 반영
        for i, data in enumerate(train_dataset):
            data.x = torch.tensor(scaled_features_train[i], dtype=torch.float).view(1, -1)
        for i, data in enumerate(valid_dataset):
            data.x = torch.tensor(scaled_features_valid[i], dtype=torch.float).view(1, -1)
        for i, data in enumerate(test_dataset):
            data.x = torch.tensor(scaled_features_test[i], dtype=torch.float).view(1, -1)       

    train_loader = loader_dataset(data_list=train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = loader_dataset(data_list=valid_dataset, batch_size=batch_size, shuffle=False) 
    test_loader = loader_dataset(data_list=test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, valid_loader, test_loader

`dataset_name` : 불러올 데이터셋의 이름 입력

`data_type` : Chapter2 에서 저장한 데이터셋 유형 (Token, Fingerprint, Descriptors, Graph) 중 하나를 선택

`batch_size` : 한번에 처리할 샘플 개수를 지정

 Graph loader가 어떻게 구성되어 있는지 확인해본다

### Graph Loader

In [5]:
train_loader, valid_loader, test_loader = get_dataloader(dataset_name = 'bace', data_type='Graph', batch_size = 128)
for data in train_loader:
    print(data)
    break

DataBatch(x=[4311, 9], edge_index=[2, 9322], edge_attr=[9322, 3], smiles=[128], y=[128, 1], batch=[4311], ptr=[129])


- x: 그래프의 노드 feature
- edge_index: 그래프의 두 노드 간 연결(edge) 정보
- edge_attr : 노드 간 연결(edge)의 feature
- smiles : 그래프가 표현하는 분자의 smiles
- y : task에 대한 label
- batch : batch 내의 총 노드 개수
- ptr : 각 그래프의 시작 노드의 위치. 그래프 개수(batch size) + 1 의 길이이며, 마지막 값은 batch에 포함된 전체 노드의 개수를 의미.

다른 표현형들의 경우 feature는 x에, label은 y에 저장되어있다.

## 2) 모델 제작

분자의 표현형에 따라 데이터 타입이 달라지므로, 각기 다른 모델을 사용하여야 한다.

예를 들어 Graph 형태의 데이터는 Graph를 처리할 수 있는 GNN을, String Tokenization 형태의 경우 순서 정보를 반영할 수 있는 1D CNN, RNN 등을 사용할 수 있다.

가장 기본적인 모델인 MLP를 먼저 사용해보자


1. Fingerprint

- Fingerprint는 분자의 특성을 단순히 vector 형태로 나타낸 것이다.
- 따라서 기본 모델인 MLP로 학습하는 것이 적합하다.
- 아래에 정의한 MLP 차원 및 layer 개수는 예시이므로, 데이터셋에 적합하게 수정하여 사용한다.

In [7]:
# 모델 
class MLP(nn.Module): 
    def __init__(self, input_dim, hidden_dim, num_classes, dropout_rate=0.5): 
        super(MLP, self).__init__() 
        self.layer1 = nn.Linear(input_dim, hidden_dim) # input feature가 4개 
        self.layer2 = nn.Linear(hidden_dim,hidden_dim*2)
        self.layer3 = nn.Linear(hidden_dim*2,hidden_dim*1)
        self.layer4 = nn.Linear(hidden_dim*1, num_classes) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self,data): # 모델에서 실행되어야 하는 계산을 정의
        x = self.layer1(data.x.float()) # data의 feature가 들어있는 x 만 받아옴, 연산 시 데이터 타입을 일치시키기 위해 float() 
        x = self.relu(x)
        x = self.dropout(x)  
        x = self.layer2(x)
        x = self.relu(x)
        x = self.dropout(x)  
        x = self.layer3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.layer4(x)   # 출력층
        return x

- `input dim` : 모델의 입력 차원을 의미하며, 데이터 feature의 차원에 따라 결정되므로 적용하려는 데이터에 맞게 설정한다. 

- `hidden dim` : 입력 층을 거친 후 은닉층의 차원을 설정하는 부분이다. 이 파라미터와 layer 개수에 따라 모델의 capacity가 결정된다.

- `num_classes` : 분류할 클래스의 개수를 나타낸다.

- `dropout_rate` : dropout은 일부 뉴런을 랜덤으로 비활성화 하여 과적합을 방지하기 위한 기법이다. 이 파라미터는 비활성화 할 뉴런의 비율을 결정한다.

- 각 레이어 사이에 비선형성을 추가하기 위해 활성화 함수가 포함된다.(예시에서는 ReLU사용)




학습 전, 입력 차원을 결정하기 위해 data loader에서 모델에 학습할 정보를 담고 있는 x의 shape를 확인한다.

In [8]:
# Fingerprint Loader 확인
train_loader, valid_loader, test_loader = get_dataloader(dataset_name = 'bace', data_type='Fingerprint', batch_size = 128)
for data in train_loader:
    print(data)
    break

DataBatch(x=[128, 1024], y=[128, 1], smiles=[128], batch=[128], ptr=[129])


데이터의 입력을 나타내는 x의 차원이 1024이므로 `input_dim` = 1024가 된다. 또한 bace는 이진 분류이므로 `num_classes` = 1로 설정할 수 있다.

In [9]:
fp_model1 = MLP(input_dim=1024, hidden_dim = 128, num_classes = 1, dropout_rate=0.5)
fp_model1

MLP(
  (layer1): Linear(in_features=1024, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=256, bias=True)
  (layer3): Linear(in_features=256, out_features=128, bias=True)
  (layer4): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
)

임의로 설정된 `hidden_dim` 및 `dropout_rate`을 자유롭게 설정하여 모델을 만들어보자

In [10]:
fp_model2 = MLP(input_dim=1024, hidden_dim = 32, num_classes = 1, dropout_rate=0.7)
fp_model2

MLP(
  (layer1): Linear(in_features=1024, out_features=32, bias=True)
  (layer2): Linear(in_features=32, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=32, bias=True)
  (layer4): Linear(in_features=32, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.7, inplace=False)
)

2. Descriptors

- Descriptors 역시 분자의 특성을 vector 형태로 나타낸 것이므로 MLP를 사용하여 학습한다.
- 사전 정의한 MLP 모델에서 입력 차원만을 변경하여 사용한다.

In [11]:
# Descriptors Loader 확인
train_loader, valid_loader, test_loader = get_dataloader(dataset_name = 'bace', data_type='Descriptors', batch_size = 128)
for data in train_loader:
    print(data)
    break

DataBatch(x=[128, 210], y=[128, 1], smiles=[128], batch=[128], ptr=[129])


descriptor의 x 차원은 210이므로 `input_dim` = 210이 된다.

In [12]:
ds_model = MLP(input_dim=210, hidden_dim = 128, num_classes = 1, dropout_rate=0.2)
ds_model

MLP(
  (layer1): Linear(in_features=210, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=256, bias=True)
  (layer3): Linear(in_features=256, out_features=128, bias=True)
  (layer4): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
)

3. Token

- Token은 분자를 문자열로 나타낸 데이터 형태이다.
- 따라서 문자열의 순서를 반영할 수 있는 1D CNN을 사용하여 학습해본다.

In [13]:
class Conv1d(nn.Module):
    def __init__(self, vocab_num, seq_len, emb_dim, hidden_size, kernel_size, num_classes):
        super(Conv1d, self).__init__()
        
        # init
        self.relu = nn.ReLU()
        self.n_filters = hidden_size
        
        # Embedding Layer
        self.embedding = nn.Embedding(vocab_num, emb_dim)
        
        # Conv1d Layers
        self.layer1 = nn.Conv1d(in_channels=seq_len, out_channels=hidden_size, kernel_size=kernel_size)
        self.layer2 = nn.Conv1d(in_channels=hidden_size, out_channels=hidden_size * 2, kernel_size=kernel_size)
        self.layer3 = nn.Conv1d(in_channels=hidden_size * 2, out_channels=hidden_size * 3, kernel_size=kernel_size)
        
        # Pooling Layer
        self.pool = nn.AdaptiveAvgPool1d(1)
        
        # Fully Connected Layer
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size * 3, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes),
        )
        
        
    def forward(self, data):
        # Embedding Layer
        x = self.embedding(data.x)  # Input shape: (batch_size, seq_len)
        
        # Conv1d Layers
        x = self.relu(self.layer1(x))  # (batch_size, hidden_size, new_seq_len)
        x = self.relu(self.layer2(x))  # (batch_size, hidden_size * 2, smaller_seq_len)
        x = self.relu(self.layer3(x))  # (batch_size, hidden_size * 3, more smaller_seq_len)
        
        # Pooling & Flatten
        x = self.pool(x).view(-1, self.n_filters * 3) # (batch, hidden_size * 3)
        
        # Fully Connected Layer
        x = self.classifier(x)  # (batch_size, 1)
        
        return x

- `vocab_num` : 문자열을 구성하는 단어의 종류를 입력한다. chapter2에서 string tokenization 단계에서 출력한 단어집에서 단어의 종류를 알 수 있다. bace의 경우 Unkown 포함 39개가 있다.

- `emb_dim` : 입력된 단어가 임베딩 될 차원을 결정한다.

- `hidden_dim` : 임베딩 된 단어를 학습 시킬 때 은닉층의 차원을 설정한다.

- `kernel_size` : kernel이 한 번에 읽을 시퀀스의 길이를 정의하며, 작을 수록 세밀한 패턴을, 클 수록 넓은 문맥을 포착한다.

- `num_classes` : 분류할 클래스의 개수를 나타낸다.

In [14]:
# Token Loader 확인
train_loader, valid_loader, test_loader = get_dataloader(dataset_name = 'bace', data_type='Token', batch_size = 128)
for data in train_loader:
    print(data)
    break

DataBatch(x=[128, 178], y=[128, 1], smiles=[128], batch=[128], ptr=[129])


In [15]:
model = Conv1d(vocab_num=39, seq_len=178, emb_dim=128, hidden_size = 128, kernel_size = 1, num_classes=1)
model

Conv1d(
  (relu): ReLU()
  (embedding): Embedding(39, 128)
  (layer1): Conv1d(178, 128, kernel_size=(1,), stride=(1,))
  (layer2): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
  (layer3): Conv1d(256, 384, kernel_size=(1,), stride=(1,))
  (pool): AdaptiveAvgPool1d(output_size=1)
  (classifier): Sequential(
    (0): Linear(in_features=384, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=512, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): ReLU()
    (8): Linear(in_features=256, out_features=1, bias=True)
  )
)

4. Graph

- Graph는 분자를 구성하는 각 원자를 노드(node), 원자들의 결합을 엣지(edge)로 나타낸 데이터 형태이다.
- Graph 형태의 데이터를 처리할 수 있는 신경망인 Graph Neural Network(GNN)을 사용하여야 한다.
- 이번 예시에서는 GNN의 예시 중 하나인 GIN을 구현해볼 것이다.

- GNN의 입력에는 노드의 특징 차원이 필요하다. data loader를 print하여 노드 x의 feature 차원을 출력해보자

In [16]:
# Descriptors Loader 확인
train_loader, valid_loader, test_loader = get_dataloader(dataset_name = 'bace', data_type='Graph', batch_size = 128)
for data in train_loader:
        print(data)
        break

DataBatch(x=[4355, 9], edge_index=[2, 9442], edge_attr=[9442, 3], smiles=[128], y=[128, 1], batch=[4355], ptr=[129])


- GNN은 각 노드들의 정보 전파 및 종합이 일어나는 gnn layer와, gnn layer가 생성한 표현을 바탕으로 분류 및 회귀 작업이 일어나는 feed forward network(ffn)로 구성된다. 따라서 `hidden_dim`은 gnn의 은닉층 차원을, `ffn_hidden`은 ffn layer의 은닉층의 차원을 지정하는 파라미터이다.
- `input_dim` : 노드 feature의 차원을 입력한다.
- `num_classes`: ffn layer의 마지막 차원을 결정하는 파라미터이므로, 분류하고자 하는 클래스의 개수를 입력한다.

In [17]:
class GINConvNet(torch.nn.Module):
    """Graph Isomorphism Network class with 3 GINConv layers and 2 linear layers"""

    def __init__(self, input_dim, dim_h, dropout=0.2):
        """Initializing GIN class

        Args:
            dim_h (int): the dimension of hidden layers
        """
        super(GINConvNet, self).__init__()
        self.conv1 = GINConv(
            Sequential(Linear(input_dim, dim_h), nn.BatchNorm1d(dim_h), ReLU(), Linear(dim_h, dim_h), ReLU())
        )
        self.conv2 = GINConv(
            Sequential(
                Linear(dim_h, dim_h), nn.BatchNorm1d(dim_h), ReLU(), Linear(dim_h, dim_h), ReLU()
            )
        )
        self.conv3 = GINConv(
            Sequential(
                Linear(dim_h, dim_h), nn.BatchNorm1d(dim_h), ReLU(), Linear(dim_h, dim_h), ReLU()
            )
        )
        self.lin1 = Linear(dim_h, dim_h)
        self.lin2 = Linear(dim_h, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, data):
        x = data.x
        edge_index = data.edge_index
        batch = data.batch

        # Node embeddings
        h = self.conv1(x, edge_index)
        h = h.relu()
        h = self.conv2(h, edge_index)
        h = h.relu()
        h = self.conv3(h, edge_index)

        # Graph-level readout
        h = gap(h, batch)

        h = self.lin1(h)
        h = h.relu()
        h = self.dropout(h)
        h = self.lin2(h)

        return h

In [18]:
g_model1 = GINConvNet(input_dim=9, dim_h=128)
g_model1

GINConvNet(
  (conv1): GINConv(nn=Sequential(
    (0): Linear(in_features=9, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=128, bias=True)
    (4): ReLU()
  ))
  (conv2): GINConv(nn=Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=128, bias=True)
    (4): ReLU()
  ))
  (conv3): GINConv(nn=Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=128, bias=True)
    (4): ReLU()
  ))
  (lin1): Linear(in_features=128, out_features=128, bias=True)
  (lin2): Linear(in_features=128, out_features=1, bias=True)
  (dropout)

## 3) 학습 및 평가


In [18]:
# 학습 진행 함수 정의
device = "cuda" if torch.cuda.is_available() else "cpu"

def train(model, train_loader, valid_loader, epochs):
    model = model.to(device)
    # train_loader, valid_loader, test_loader = loaders
    # optimizer = torch.optim.SGD(model.parameters(), lr = 0.00001, momentum = 0.5) 
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001) 
    criterion = nn.BCEWithLogitsLoss() 

    best_valid_loss = float('inf')  # 초기값을 무한대로 설정
    best_epoch = 0  # 가장 성능 좋은 epoch을 기록

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        train_loss = 0.0
        model.train() # 학습 모드 전환    

        # train
        for data in train_loader:
            data = data.to(device)
            y = data.y
            
            optimizer.zero_grad() # 전 단계에서의 loss gradient 값을 초기화
            output = model(data)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        print(f"Training Loss: {train_loss / len(train_loader):.4f}")

        # validation
        model.eval() # 학습 모드 전환    
        valid_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():  # Gradient 계산 비활성화
            for data in valid_loader:
                data = data.to(device)
                y = data.y
                
                output = model(data)
                loss = criterion(output, y)
                valid_loss += loss.item()

                # Accuracy 계산
                predicted = (F.sigmoid(output) > 0.5).int()

                total += y.size(0)
                correct += (predicted == y).sum().item()

        accuracy = 100 * (correct / total)
        avg_valid_loss = valid_loss / len(valid_loader)
        print(f"Validation Loss: {avg_valid_loss:.4f}, Validation Accuracy: {accuracy:.2f}%\n")
        
        # Best model 저장
        if avg_valid_loss < best_valid_loss:
            best_valid_loss = avg_valid_loss
            best_epoch = epoch + 1
            torch.save(model.state_dict(), "best_epoch.pth")
            print(f"Best model updated at epoch {best_epoch} with validation loss: {best_valid_loss:.4f}")

    print(f"Training completed. Best model was at epoch {best_epoch} with validation loss: {best_valid_loss:.4f}")
    

- `model` : 학습할 데이터셋에 맞게 설정한 모델 입력
- `train_loader`, `valid_loader` : 학습, 검증 data loader 입력
- `epochs` : 반복할 학습 횟수 입력

In [19]:
# fingerprint loader 생성
train_loader, valid_loader, test_loader = get_dataloader(dataset_name = 'bace', data_type='Fingerprint', batch_size = 128)

# 학습할 모델 정의
fp_model1 = MLP(input_dim=1024, hidden_dim = 128, num_classes = 1, dropout_rate=0.2)

# train
p = train(fp_model1, train_loader, valid_loader, epochs=10)

Epoch 1/10
Training Loss: 0.6828
Validation Loss: 0.8653, Validation Accuracy: 13.91%

Best model updated at epoch 1 with validation loss: 0.8653
Epoch 2/10
Training Loss: 0.5582
Validation Loss: 0.9162, Validation Accuracy: 48.34%

Epoch 3/10
Training Loss: 0.4381
Validation Loss: 0.6916, Validation Accuracy: 65.56%

Best model updated at epoch 3 with validation loss: 0.6916
Epoch 4/10
Training Loss: 0.3413
Validation Loss: 0.6377, Validation Accuracy: 60.93%

Best model updated at epoch 4 with validation loss: 0.6377
Epoch 5/10
Training Loss: 0.2788
Validation Loss: 1.0373, Validation Accuracy: 51.66%

Epoch 6/10
Training Loss: 0.2538
Validation Loss: 0.9639, Validation Accuracy: 53.64%

Epoch 7/10
Training Loss: 0.2261
Validation Loss: 0.8928, Validation Accuracy: 55.63%

Epoch 8/10
Training Loss: 0.1881
Validation Loss: 0.7465, Validation Accuracy: 60.26%

Epoch 9/10
Training Loss: 0.1571
Validation Loss: 0.9983, Validation Accuracy: 56.29%

Epoch 10/10
Training Loss: 0.1226
Valida

In [20]:
# Descriptor loader 생성
train_loader, valid_loader, test_loader = get_dataloader(dataset_name = 'bace', data_type='Descriptors', batch_size = 128)

# 학습할 모델 정의
ds_model = MLP(input_dim=210, hidden_dim = 128, num_classes = 1, dropout_rate=0.2)

# train
train(ds_model, train_loader, valid_loader, epochs=50)

Epoch 1/50
Training Loss: 0.6813
Validation Loss: 0.9569, Validation Accuracy: 13.91%

Best model updated at epoch 1 with validation loss: 0.9569
Epoch 2/50
Training Loss: 0.6542
Validation Loss: 0.9008, Validation Accuracy: 13.91%

Best model updated at epoch 2 with validation loss: 0.9008
Epoch 3/50
Training Loss: 0.6061
Validation Loss: 0.9626, Validation Accuracy: 32.45%

Epoch 4/50
Training Loss: 0.5361
Validation Loss: 1.1104, Validation Accuracy: 45.70%

Epoch 5/50
Training Loss: 0.4948
Validation Loss: 0.7691, Validation Accuracy: 66.89%

Best model updated at epoch 5 with validation loss: 0.7691
Epoch 6/50
Training Loss: 0.4820
Validation Loss: 0.7187, Validation Accuracy: 68.87%

Best model updated at epoch 6 with validation loss: 0.7187
Epoch 7/50
Training Loss: 0.4548
Validation Loss: 0.7487, Validation Accuracy: 66.23%

Epoch 8/50
Training Loss: 0.4291
Validation Loss: 0.8553, Validation Accuracy: 64.24%

Epoch 9/50
Training Loss: 0.4111
Validation Loss: 0.6007, Validation

In [21]:
# Token loader 생성
train_loader, valid_loader, test_loader = get_dataloader(dataset_name = 'bace', data_type='Token', batch_size = 128)

# 학습할 모델 정의
token_model = Conv1d(vocab_num=39, seq_len=178, emb_dim=64, hidden_size=32, kernel_size = 4, num_classes=1)

# train
train(token_model, train_loader, valid_loader, epochs=10)

Epoch 1/10
Training Loss: 0.6767
Validation Loss: 0.8005, Validation Accuracy: 13.91%

Best model updated at epoch 1 with validation loss: 0.8005
Epoch 2/10
Training Loss: 0.6638
Validation Loss: 1.0546, Validation Accuracy: 13.91%

Epoch 3/10
Training Loss: 0.6584
Validation Loss: 0.8463, Validation Accuracy: 13.91%

Epoch 4/10
Training Loss: 0.6280
Validation Loss: 0.9499, Validation Accuracy: 13.91%

Epoch 5/10
Training Loss: 0.5714
Validation Loss: 0.8492, Validation Accuracy: 54.30%

Epoch 6/10
Training Loss: 0.5368
Validation Loss: 0.6796, Validation Accuracy: 78.81%

Best model updated at epoch 6 with validation loss: 0.6796
Epoch 7/10
Training Loss: 0.5220
Validation Loss: 1.0238, Validation Accuracy: 45.70%

Epoch 8/10
Training Loss: 0.4587
Validation Loss: 0.6952, Validation Accuracy: 73.51%

Epoch 9/10
Training Loss: 0.4533
Validation Loss: 0.6887, Validation Accuracy: 71.52%

Epoch 10/10
Training Loss: 0.4373
Validation Loss: 0.7739, Validation Accuracy: 66.89%

Training co

In [22]:
# Graph loader 생성
train_loader, valid_loader, test_loader = get_dataloader(dataset_name = 'bace', data_type='Graph', batch_size = 128)

# 학습할 모델 정의
g_model = GINConvNet(input_dim=9, dim_h=128)

# train
train(g_model, train_loader, valid_loader, epochs=10)

Epoch 1/10
Training Loss: 0.8221
Validation Loss: 2.1389, Validation Accuracy: 13.91%

Best model updated at epoch 1 with validation loss: 2.1389
Epoch 2/10
Training Loss: 0.6059
Validation Loss: 1.1479, Validation Accuracy: 25.17%

Best model updated at epoch 2 with validation loss: 1.1479
Epoch 3/10
Training Loss: 0.5422
Validation Loss: 0.8822, Validation Accuracy: 47.68%

Best model updated at epoch 3 with validation loss: 0.8822
Epoch 4/10
Training Loss: 0.5047
Validation Loss: 1.1775, Validation Accuracy: 37.75%

Epoch 5/10
Training Loss: 0.4846
Validation Loss: 0.6652, Validation Accuracy: 57.62%

Best model updated at epoch 5 with validation loss: 0.6652
Epoch 6/10
Training Loss: 0.4565
Validation Loss: 1.1116, Validation Accuracy: 42.38%

Epoch 7/10
Training Loss: 0.4546
Validation Loss: 2.0816, Validation Accuracy: 19.87%

Epoch 8/10
Training Loss: 0.4367
Validation Loss: 1.0589, Validation Accuracy: 50.33%

Epoch 9/10
Training Loss: 0.4314
Validation Loss: 0.3471, Validation

학습된 모델로 평가를 수행한다.

평가는 accuracy, precision, recall, f1_score, roc_auc, average precision 총 6가지로 이루어지며, 설명은 다음과 같다.

- Accuracy (정확도): 올바르게 예측한 샘플의 비율

- Precision (정밀도): 모델이 양성으로 예측한 샘플 중 실제로 양성인 비율.

- Recall (재현율): 실제 양성 샘플 중 모델이 양성으로 예측한 비율.

- F1-Score: Precision과 Recall의 조화 평균.데이터가 불균형할 때 모델의 전반적 성능을 파악하기 좋음.

- AUC_ROC (ROC 곡선 아래 면적): 민감도(TPR)와 특이도(1-FPR) 간의 균형을 평가.1에 가까울수록 좋음.

- AUC_PRC (PRC 곡선 아래 면적): Precision과 Recall 간의 관계를 시각화한 곡선의 면적.

In [19]:
def test(model, test_loader, save_file = 'Results.csv'):
    model = model.to(device)
    model.eval()    
    criterion = nn.BCEWithLogitsLoss() 
    
    all_preds = []
    all_targets = []
    test_loss = 0.0
    
    with torch.no_grad():  # Gradient 계산 비활성화
        for data in test_loader:
            data = data.to(device)
            y = data.y
            
            output = model(data)
            preds = torch.sigmoid(output).cpu().numpy().ravel()  # Sigmoid로 확률 변환 후 numpy로 변환
            all_preds.append(preds)
            all_targets.append(y.cpu().numpy())
            
            # 손실 계산
            test_loss += criterion(output, y).item()
            
        # Concatenate predictions and targets
        all_preds = np.concatenate(all_preds)
        all_targets = np.concatenate(all_targets).astype(int)
    
        # 메트릭 계산
        test_acc = accuracy_score(all_targets, (all_preds >= 0.5).astype(int))
        test_precision = precision_score(all_targets, (all_preds >= 0.5).astype(int))
        test_recall = recall_score(all_targets, (all_preds >= 0.5).astype(int))
        test_f1 = f1_score(all_targets, (all_preds >= 0.5).astype(int))
        test_auc_roc = roc_auc_score(all_targets, all_preds)
        test_auc_prc = average_precision_score(all_targets, all_preds)
    
        # 평균 손실 계산
        test_loss /= len(test_loader)
    
        # 결과 저장
        metrics = {
            "Loss": test_loss,
            "Accuracy": test_acc,
            "Precision": test_precision,
            "Recall": test_recall,
            "F1-Score": test_f1,
            "AUC_ROC": test_auc_roc,
            "AUC_PRC": test_auc_prc
        }
    
        # 결과 출력
        print("\n최적 모델의 테스트 데이터 성능:")
        print(tabulate(pd.DataFrame(metrics, index=["Metric Value"]).T, headers="keys", tablefmt="fancy_grid"))
    
        # CSV 파일 저장
        df = pd.DataFrame(metrics, index=[0])
        df.to_csv(save_file, index=False)
        print(f"\nResults saved to {save_file}")

    return metrics

In [24]:
# best model load
g_model = GINConvNet(input_dim=9, dim_h=128)
g_model.load_state_dict(torch.load("best_epoch.pth"))
test(g_model, test_loader)


최적 모델의 테스트 데이터 성능:
╒═══════════╤════════════════╕
│           │   Metric Value │
╞═══════════╪════════════════╡
│ Loss      │       0.400567 │
├───────────┼────────────────┤
│ Accuracy  │       0.789474 │
├───────────┼────────────────┤
│ Precision │       0.742574 │
├───────────┼────────────────┤
│ Recall    │       0.925926 │
├───────────┼────────────────┤
│ F1-Score  │       0.824176 │
├───────────┼────────────────┤
│ AUC_ROC   │       0.841071 │
├───────────┼────────────────┤
│ AUC_PRC   │       0.819085 │
╘═══════════╧════════════════╛

Results saved to Results.csv


{'Loss': 0.4005669206380844,
 'Accuracy': 0.7894736842105263,
 'Precision': 0.7425742574257426,
 'Recall': 0.9259259259259259,
 'F1-Score': 0.8241758241758241,
 'AUC_ROC': 0.8410711180664232,
 'AUC_PRC': 0.8190849653900989}

### ESOL 데이터로 Regression 예측 해보기

In [36]:
from sklearn.metrics import r2_score

device = "cuda" if torch.cuda.is_available() else "cpu"

def train_reg(model, train_loader, valid_loader, epochs):
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    best_valid_rmse = float('inf')  # RMSE 기준으로 변경
    best_epoch = 0

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        
        # Training phase
        model.train()
        train_loss = 0.0
        for data in train_loader:
            data = data.to(device)
            y = data.y
            
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)
        train_rmse = np.sqrt(avg_train_loss)  # MSE를 RMSE로 변환
        print(f"Training RMSE: {train_rmse:.4f}")

        # Validation phase
        model.eval()
        valid_loss = 0.0
        all_preds = []
        all_targets = []
        
        with torch.no_grad():
            for data in valid_loader:
                data = data.to(device)
                y = data.y
                
                output = model(data)
                loss = criterion(output, y)
                valid_loss += loss.item()
                
                # Store predictions and targets
                all_preds.extend(output.cpu().numpy())
                all_targets.extend(y.cpu().numpy())

        # Calculate metrics
        avg_valid_loss = valid_loss / len(valid_loader)
        valid_rmse = np.sqrt(avg_valid_loss)  # MSE를 RMSE로 변환
        r2 = r2_score(all_targets, all_preds)
        
        print(f"Validation RMSE: {valid_rmse:.4f}, R2 Score: {r2:.4f}\n")
        
        # Save best model (RMSE 기준으로 변경)
        if valid_rmse < best_valid_rmse:
            best_valid_rmse = valid_rmse
            best_epoch = epoch + 1
            torch.save(model.state_dict(), "best_epoch.pth")
            print(f"Best model updated at epoch {best_epoch} with validation RMSE: {best_valid_rmse:.4f}")

    print(f"Training completed. Best model was at epoch {best_epoch} with validation RMSE: {best_valid_rmse:.4f}")

In [37]:
# Graph loader 생성
train_loader, valid_loader, test_loader = get_dataloader(dataset_name = 'esol', data_type='Fingerprint', batch_size = 128)

# 학습할 모델 정의
fp_reg_model = MLP(input_dim=1024, hidden_dim = 128, num_classes = 1, dropout_rate=0.2)

# train
train_reg(fp_reg_model, train_loader, valid_loader, epochs=100)

Epoch 1/100
Training RMSE: 3.5534
Validation RMSE: 3.2482, R2 Score: -1.3439

Best model updated at epoch 1 with validation RMSE: 3.2482
Epoch 2/100
Training RMSE: 2.8071
Validation RMSE: 2.1644, R2 Score: -0.0407

Best model updated at epoch 2 with validation RMSE: 2.1644
Epoch 3/100
Training RMSE: 2.0501
Validation RMSE: 1.7339, R2 Score: 0.3321

Best model updated at epoch 3 with validation RMSE: 1.7339
Epoch 4/100
Training RMSE: 1.7549
Validation RMSE: 1.5802, R2 Score: 0.4453

Best model updated at epoch 4 with validation RMSE: 1.5802
Epoch 5/100
Training RMSE: 1.4476
Validation RMSE: 1.3852, R2 Score: 0.5737

Best model updated at epoch 5 with validation RMSE: 1.3852
Epoch 6/100
Training RMSE: 1.4015
Validation RMSE: 1.2666, R2 Score: 0.6436

Best model updated at epoch 6 with validation RMSE: 1.2666
Epoch 7/100
Training RMSE: 1.3221
Validation RMSE: 1.1993, R2 Score: 0.6805

Best model updated at epoch 7 with validation RMSE: 1.1993
Epoch 8/100
Training RMSE: 1.0920
Validation R

In [38]:
# Descriptor loader 생성
train_loader, valid_loader, test_loader = get_dataloader(dataset_name = 'esol', data_type='Descriptors', batch_size = 128)

# 학습할 모델 정의
ds_reg_model = MLP(input_dim=210, hidden_dim = 128, num_classes = 1, dropout_rate=0.2)

# train
train_reg(ds_reg_model, train_loader, valid_loader, epochs=100)

Epoch 1/100
Training RMSE: 3.7181
Validation RMSE: 3.0301, R2 Score: -1.0397

Best model updated at epoch 1 with validation RMSE: 3.0301
Epoch 2/100
Training RMSE: 2.4356
Validation RMSE: 2.0624, R2 Score: 0.0551

Best model updated at epoch 2 with validation RMSE: 2.0624
Epoch 3/100
Training RMSE: 1.8898
Validation RMSE: 1.7615, R2 Score: 0.3107

Best model updated at epoch 3 with validation RMSE: 1.7615
Epoch 4/100
Training RMSE: 1.7656
Validation RMSE: 1.4981, R2 Score: 0.5014

Best model updated at epoch 4 with validation RMSE: 1.4981
Epoch 5/100
Training RMSE: 1.5916
Validation RMSE: 1.2780, R2 Score: 0.6371

Best model updated at epoch 5 with validation RMSE: 1.2780
Epoch 6/100
Training RMSE: 1.4643
Validation RMSE: 1.0944, R2 Score: 0.7339

Best model updated at epoch 6 with validation RMSE: 1.0944
Epoch 7/100
Training RMSE: 1.1111
Validation RMSE: 0.9754, R2 Score: 0.7886

Best model updated at epoch 7 with validation RMSE: 0.9754
Epoch 8/100
Training RMSE: 1.0893
Validation RM

In [42]:
# Token loader 생성
train_loader, valid_loader, test_loader = get_dataloader(dataset_name = 'esol', data_type='Token', batch_size = 128)

# 학습할 모델 정의
token_reg_model = Conv1d(vocab_num=39, seq_len=97, emb_dim=64, hidden_size=32, kernel_size = 4, num_classes=1)

# train
train_reg(token_reg_model, train_loader, valid_loader, epochs=100)

Epoch 1/10
Training RMSE: 0.6134
Validation RMSE: 0.5051, R2 Score: -0.1112

Best model updated at epoch 1 with validation RMSE: 0.5051
Epoch 2/10
Training RMSE: 0.5003
Validation RMSE: 0.4959, R2 Score: -0.0709

Best model updated at epoch 2 with validation RMSE: 0.4959
Epoch 3/10
Training RMSE: 0.4955
Validation RMSE: 0.4795, R2 Score: -0.0015

Best model updated at epoch 3 with validation RMSE: 0.4795
Epoch 4/10
Training RMSE: 0.4893
Validation RMSE: 0.4826, R2 Score: -0.0145

Epoch 5/10
Training RMSE: 0.4832
Validation RMSE: 0.4796, R2 Score: -0.0019

Epoch 6/10
Training RMSE: 0.4820
Validation RMSE: 0.4809, R2 Score: -0.0073

Epoch 7/10
Training RMSE: 0.4924
Validation RMSE: 0.4808, R2 Score: -0.0068

Epoch 8/10
Training RMSE: 0.4982
Validation RMSE: 0.4862, R2 Score: -0.0297

Epoch 9/10
Training RMSE: 0.4897
Validation RMSE: 0.4802, R2 Score: -0.0043

Epoch 10/10
Training RMSE: 0.4925
Validation RMSE: 0.4837, R2 Score: -0.0189

Training completed. Best model was at epoch 3 with v

In [46]:
# Graph loader 생성
train_loader, valid_loader, test_loader = get_dataloader(dataset_name = 'esol', data_type='Graph', batch_size = 128)

# 학습할 모델 정의
g_reg_model = GINConvNet(input_dim=9, dim_h=128)

# train
train_reg(g_reg_model, train_loader, valid_loader, epochs=100)

Epoch 1/100
Training RMSE: 1.9194
Validation RMSE: 1.9734, R2 Score: 0.1349

Best model updated at epoch 1 with validation RMSE: 1.9734
Epoch 2/100
Training RMSE: 1.2923
Validation RMSE: 1.7688, R2 Score: 0.3050

Best model updated at epoch 2 with validation RMSE: 1.7688
Epoch 3/100
Training RMSE: 1.2405
Validation RMSE: 1.1164, R2 Score: 0.7231

Best model updated at epoch 3 with validation RMSE: 1.1164
Epoch 4/100
Training RMSE: 1.2084
Validation RMSE: 1.1161, R2 Score: 0.7233

Best model updated at epoch 4 with validation RMSE: 1.1161
Epoch 5/100
Training RMSE: 1.2652
Validation RMSE: 1.2352, R2 Score: 0.6610

Epoch 6/100
Training RMSE: 1.1217
Validation RMSE: 1.0227, R2 Score: 0.7676

Best model updated at epoch 6 with validation RMSE: 1.0227
Epoch 7/100
Training RMSE: 1.0520
Validation RMSE: 1.2283, R2 Score: 0.6648

Epoch 8/100
Training RMSE: 1.1259
Validation RMSE: 0.8630, R2 Score: 0.8345

Best model updated at epoch 8 with validation RMSE: 0.8630
Epoch 9/100
Training RMSE: 1.0

### Regression 모델 테스트 결과

In [43]:
def test_regression(model, test_loader, save_file='Results_regression.csv'):
    model = model.to(device)
    model.eval()    
    criterion = nn.MSELoss()  # Changed to MSE for regression
    
    all_preds = []
    all_targets = []
    test_loss = 0.0
    
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            y = data.y
            
            output = model(data)
            preds = output.cpu().numpy().ravel()  # Removed sigmoid, direct output for regression
            all_preds.append(preds)
            all_targets.append(y.cpu().numpy())
            
            # Loss calculation
            test_loss += criterion(output, y).item()
            
        # Concatenate predictions and targets
        all_preds = np.concatenate(all_preds)
        all_targets = np.concatenate(all_targets)
    
        # Calculate regression metrics
        mse = np.mean((all_preds - all_targets) ** 2)
        rmse = np.sqrt(mse)
        mae = np.mean(np.abs(all_preds - all_targets))
        r2 = r2_score(all_targets, all_preds)
    
        # Average loss
        test_loss /= len(test_loader)
    
        # Store metrics
        metrics = {
            "MSE": mse,
            "RMSE": rmse,
            "MAE": mae,
            "R²": r2
        }
    
        # Print results
        print("\nTest Performance Metrics:")
        print(tabulate(pd.DataFrame(metrics, index=["Metric Value"]).T, 
                      headers="keys", 
                      tablefmt="fancy_grid"))

    return metrics

In [47]:
# Graph loader 생성
train_loader, valid_loader, test_loader = get_dataloader(dataset_name = 'esol', data_type='Graph', batch_size = 128)
g_reg_model = GINConvNet(input_dim=9, dim_h=128)
g_reg_model.load_state_dict(torch.load("best_epoch.pth"))
test_regression(g_reg_model, test_loader)


Test Performance Metrics:
╒══════╤════════════════╕
│      │   Metric Value │
╞══════╪════════════════╡
│ MSE  │       7.68855  │
├──────┼────────────────┤
│ RMSE │       2.77282  │
├──────┼────────────────┤
│ MAE  │       2.1769   │
├──────┼────────────────┤
│ R²   │       0.836798 │
╘══════╧════════════════╛


{'MSE': 7.688552,
 'RMSE': 2.7728238,
 'MAE': 2.1769023,
 'R²': 0.8367981910705566}